In [ ]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter("ignore")
import time
from itertools import groupby
import os
import random
import json
import datetime
from aiogram import Bot, Dispatcher, executor, types
from aiogram.dispatcher.filters import Text
from aiogram.utils.markdown import hbold,hlink
import markup as nav
import nest_asyncio

token='' #your bot token here


def get_main_categories(url):
    types_list=[]
    response=requests.get(url=url)
    soup=BeautifulSoup(response.text,'lxml')
    links=soup.find_all('ul',class_='tn-type-materials')
    new=links[0].find_all('a')
    for i in new:
        types_list.append(str('https://tengrinews.kz/')+str(i.get('href')))
    with open(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\tengrinews\types.txt','w',encoding='utf-8') as f:
        for element in types_list:
            f.write("%s\n" % element) 
            
            
def get_learn_news(file_path,type_news):
    with open(file_path) as file:
        url_list=file.readlines()
        clear_url=[]
        for url in url_list:
            clear_url.append(url.strip())
    result=[]        
    for i in range(1,10):
        data_news=[]
        title_news=[]
        link_news=[]
        response=requests.get(clear_url[0]+str('/page/')+str(i))
        soup=BeautifulSoup(response.text,'lxml')
        title=soup.find_all('span',class_='tn-article-title')
        for i in title:
            title_news.append(i.text.strip())                
        data=soup.find_all('time')
        for j in data:
            data_news.append(j.text.strip())                
        link=soup.find_all('a',class_='tn-link')
        for i in link:
            link_news.append(str('https://tengrinews.kz/')+str(i.get('href')))           
        for k in range(len(title_news)):
            result.append({
                'Категория':'УЗНАЙ',                    
                'id':[int(''.join(group)) for key, group in groupby(iterable=link_news[k], key=lambda e: e.isdigit()) if key][0],
                'Название':title_news[k],
                'Дата':data_news[k],                    
                'Ссылка':link_news[k]               
            })            
               
    result_today=[]
    for k in result:
        if (k.get('Дата').split(',')[0]==type_news):
            result_today.append(k)    
    with open(f"C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\tengrinews\\learn_{type_news}.json",'w',encoding='utf-8') as file:
        json.dump(result_today,file,indent=4,ensure_ascii=False)
        
def get_read_news(file_path,type_news):
    with open(file_path) as file:
        url_list=file.readlines()
        clear_url=[]
        for url in url_list:
            clear_url.append(url.strip())
    result=[]        
    for i in range(1,10):
        data_news=[]
        title_news=[]
        link_news=[]
        response=requests.get(clear_url[1]+str('/page/')+str(i))
        soup=BeautifulSoup(response.text,'lxml')
        title=soup.find_all('span',class_='tn-article-title')
        for i in title:
            title_news.append(i.text.strip())                
        data=soup.find_all('time')
        for j in data:
            data_news.append(j.text.strip())                
        link=soup.find_all('a',class_='tn-link')
        for i in link:
            link_news.append(str('https://tengrinews.kz/')+str(i.get('href')))          
        for k in range(len(title_news)):
            result.append({
                'Категория':'Почитай',                    
                'id':[int(''.join(group)) for key, group in groupby(iterable=link_news[k], key=lambda e: e.isdigit()) if key][0],
                'Название':title_news[k],
                'Дата':data_news[k],                    
                'Ссылка':link_news[k]               
            })            
               
    result_today=[]
    for k in result:
        if (k.get('Дата').split(',')[0]==type_news):
            result_today.append(k)    
    with open(f"C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\tengrinews\\read_{type_news}.json",'w',encoding='utf-8') as file:
        json.dump(result_today,file,indent=4,ensure_ascii=False)            

def get_watch_news(file_path,type_news):
    with open(file_path) as file:
        url_list=file.readlines()
        clear_url=[]
        for url in url_list:
            clear_url.append(url.strip())
    result=[]        
    for i in range(1,10):
        data_news=[]
        title_news=[]
        link_news=[]
        response=requests.get(clear_url[2]+str('/page/')+str(i))
        soup=BeautifulSoup(response.text,'lxml')
        title=soup.find_all('span',class_='tn-article-title')
        for i in title:
            title_news.append(i.text.strip())                
        data=soup.find_all('time')
        for j in data:
            data_news.append(j.text.strip())                
        link=soup.find_all('a',class_='tn-link')
        for i in link:
            link_news.append(str('https://tengrinews.kz/')+str(i.get('href')))          
        for k in range(len(title_news)):
            result.append({
                'Категория':'Посмотри',                    
                'id':[int(''.join(group)) for key, group in groupby(iterable=link_news[k], key=lambda e: e.isdigit()) if key][0],
                'Название':title_news[k],
                'Дата':data_news[k],                    
                'Ссылка':link_news[k]               
            })            
    
           
    result_today=[]
    for k in result:
        if (k.get('Дата').split(',')[0]==type_news):
            result_today.append(k)    

    with open(f"C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\tengrinews\\watch_{type_news}.json",'w',encoding='utf-8') as file:
        json.dump(result_today,file,indent=4,ensure_ascii=False)    


 
    
def telegram_bot(token):   
    
    nest_asyncio.apply()
    bot=Bot(token,parse_mode=types.ParseMode.HTML)
    dp=Dispatcher(bot)
    
    
    @dp.message_handler(commands='start')    
    async def start_message(message: types.Message):
        await bot.send_message(message.from_user.id,'Выберите категорию ниже...', reply_markup=nav.mainMenu)
        
    @dp.message_handler(Text(equals='главное'))
    async def general_news(message: types.Message):
        await bot.send_message(message.from_user.id,'главное', reply_markup=nav.mainMenu)
        
    @dp.message_handler(Text(equals='Узнай'))
    async def general_days(message: types.Message):
        await bot.send_message(message.from_user.id,'Узнай', reply_markup=nav.dayBtn)
        
    @dp.message_handler(Text(equals='Посмотри'))
    async def general_days(message: types.Message):
        await bot.send_message(message.from_user.id,'Посмотри', reply_markup=nav.dayBtn)
        
    @dp.message_handler(Text(equals='Почитай'))
    async def general_days(message: types.Message):
        await bot.send_message(message.from_user.id,'Почитай', reply_markup=nav.dayBtn)      
        
    @dp.message_handler()
    async def day(message: types.Message):        
        await message.answer('Подождите...')
        text=message.text
        if text=='сегодня' or text=='вчера':
            get_learn_news(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\tengrinews\types.txt',type_news=text)
            with open(f"C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\tengrinews\\learn_{text}.json",'r',encoding='utf-8') as f:
                    data=json.load(f)
            for index,item in enumerate(data):
                        card=f"{hbold('Дата: ')}{item.get('Дата')}\n"\
                             f"{hlink(item.get('Название'),item.get('Ссылка'))}\n"\
                             f"{hbold('Категория: ')}{item.get('Категория')}"
                        if index%5==0:
                            time.sleep(3)                        
                        await message.answer(card)
        
    executor.start_polling(dp,skip_updates=True)    
    

def main():
    
    #get_main_categories('https://tengrinews.kz/')
    #get_learn_news(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\tengrinews\types.txt',type_news='сегодня')
    telegram_bot(token)
    
if __name__=="__main__":
     main()


    
